# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell, Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144


Python None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298698742335                   -0.85    4.8         
  2   -8.300230864219       -2.81       -1.26    1.2    156ms
  3   -8.300439670218       -3.68       -1.89    3.5    195ms
  4   -8.300461629740       -4.66       -2.74    1.8    149ms
  5   -8.300464100982       -5.61       -3.08    3.0    204ms
  6   -8.300464386017       -6.55       -3.28   10.1    345ms
  7   -8.300464518655       -6.88       -3.43    7.4    266ms
  8   -8.300464583143       -7.19       -3.56    1.4    157ms
  9   -8.300464627896       -7.35       -3.77    2.0    205ms
 10   -8.300464638951       -7.96       -3.94    1.2    155ms
 11   -8.300464643806       -8.31       -4.36    1.8    231ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.64179327691                   -0.70    6.9         
  2   -16.67355260275       -1.50       -1.14    1.2    360ms
  3   -16.67919183020       -2.25       -1.87    3.4    446ms
  4   -16.67927521849       -4.08       -2.72    3.2    461ms
  5   -16.67928512779       -5.00       -3.09    4.8    551ms
  6   -16.67928617205       -5.98       -3.48    2.1    379ms
  7   -16.67928621162       -7.40       -3.93    2.2    378ms
  8   -16.67928622123       -8.02       -4.55    3.1    464ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32734968770                   -0.56    7.5         
  2   -33.33442641758       -2.15       -1.00    1.5    1.33s
  3   -33.33596626240       -2.81       -1.72    5.6    1.73s
  4   -33.33683429318       -3.06       -2.30    7.1    1.84s
  5   -33.33693077748       -4.02       -2.78    5.8    1.89s
  6   -33.33694365229       -4.89       -3.75    3.9    1.72s
  7   -33.33694375532       -6.99       -3.99    4.6    1.85s
  8   -33.33694378215       -7.57       -4.85    2.6    1.42s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298518783973                   -0.85    4.9         
  2   -8.300268941460       -2.76       -1.59    1.2    112ms
  3   -8.300429480494       -3.79       -2.58    3.4    164ms
  4   -8.300395908403   +   -4.47       -2.33    5.1    224ms
  5   -8.300464267506       -4.17       -3.38    1.0    115ms
  6   -8.300464590839       -6.49       -3.82    3.9    195ms
  7   -8.300464639820       -7.31       -4.21    2.2    182ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32702095447                   -0.56    6.9         
  2   -33.31153218392   +   -1.81       -1.27    1.5    1.10s
  3   -10.75640986532   +    1.35       -0.39    6.8    2.35s
  4   -33.33090692699        1.35       -1.83    5.6    2.10s
  5   -33.17155668775   +   -0.80       -1.27    4.0    1.93s
  6   -32.12691653339   +    0.02       -1.02    4.4    1.86s
  7   -33.32916178456        0.08       -2.00    4.2    1.68s
  8   -33.33382658455       -2.33       -2.21    2.0    1.30s
  9   -33.33631950192       -2.60       -2.40    2.5    1.43s
 10   -33.33650905407       -3.72       -2.72    1.8    1.27s
 11   -33.33687079047       -3.44       -3.08    2.9    1.35s
 12   -33.33693682531       -4.18      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.